In [2]:
import ipdb
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pylab as pylab
import cartopy
from utils import constants as cnst
from eod import msg
from utils import u_grid, u_interpolate as u_int, constants as cnst, u_arrays, u_darrays, u_met
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as feature
import cartopy.io.shapereader as shapereader
import pickle as pkl
import glob

coast = shapereader.natural_earth(resolution='110m',
                                  category='physical',
                                  name='coastline')

countries = shapereader.natural_earth(resolution='110m',
                                      category='cultural',
                                      name='admin_0_countries')

pylab.rcParams['figure.figsize'] = (50., 50.)
%matplotlib inline

In [1]:
cd ..

/home/ck/pythonWorkspace/proj_CEH


In [29]:
filetag = 'pl'
era_hour = 12

In [44]:
def save_node_clim(filetag, node, era_hour):
    
    nodestr = 'node'+str(node)
    all_node_file = pkl.load(open(cnst.network_data + 'data/SOMS/saves/node_days.p', "rb"))
    node_days = all_node_file[nodestr]
    node_days_era = []
    for nd in node_days:
        node_days_era.append(nd.replace(hour=era_hour))
    node_days_era = pd.Series(node_days_era).values
    
    if filetag == 'pl':
        fft = 'pressure_levels'
    else:
        fft = 'surface'
        
    filebase = cnst.elements_drive + '/Africa/WestAfrica/ERA5/hourly/' 
    filepath = filebase + fft+'/ERA5*'+filetag+'.nc'
    out = '/home/ck/DIR/cornkle/data/SOMS/saves/'+nodestr+'_ERA5_clim_mean_'+filetag+'.nc'
    outdev = '/home/ck/DIR/cornkle/data/SOMS/saves/'+nodestr+'_ERA5_clim_stddev_'+filetag+'.nc'
    
    def means(da, node_days):
        
        da = da.sel(time=(da['time.hour']==era_hour))
        
        try:
            da = da.sel(level=[925,850,750,650,550,250])
        except:
            pass
        #ipdb.set_trace()
        intersect_dates = np.intersect1d(node_days, da.time.values)
        #ipdb.set_trace()
        if len(intersect_dates) == 0:
            return None
        try:
            da = da.sel(time=intersect_dates)
        except:
            return None

        da = u_darrays.flip_lat(da)
        return da
    
    srfc_list = []
    
    for f in glob.glob(filepath):

        print('Doing', f)
        srfc = xr.open_dataset(f) 
        srfc_concat = means(srfc,node_days_era)
        if srfc_concat is None:
            continue
   
        srfc_list.append(srfc_concat)
    
    full = xr.concat(srfc_list, dim='time')
   
    mean = full.mean('time')
    dev = full.std('time')
    
    mean.to_netcdf(out)
    dev.to_netcdf(outdev)

In [45]:
for ni in range(1,13):
#     nodestr='node'+str(1)
#     all_node_file = pkl.load(open(cnst.network_data + 'data/SOMS/saves/node_days.p', "rb"))
#     node_days = all_node_file[nodestr]
#     node_days_era = []
#     for nd in node_days:
#         node_days_era.append(nd.replace(hour=era_hour))
        
    save_node_clim(filetag, ni, era_hour)

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1990_01_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1990_02_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1990_03_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1990_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1990_05_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1990_06_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1990_07_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1990_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1990_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1990_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1990_11_pl.nc

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2015_03_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2015_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2015_05_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2015_06_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2015_07_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2015_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2015_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2015_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2015_11_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2016_01_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2016_02_pl.nc

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2006_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2006_05_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2006_06_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2006_07_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2006_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2006_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2006_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2006_11_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2006_12_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2007_01_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2007_02_pl.nc

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1998_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1998_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1998_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1998_11_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1998_12_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1999_01_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1999_02_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1999_03_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1999_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1999_05_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1999_06_pl.nc

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2005_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2007_05_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2008_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2010_03_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2011_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2015_01_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2015_12_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2017_05_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2011_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2011_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2011_11_pl.nc

...

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2003_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2003_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2003_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2003_11_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2003_12_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2004_01_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2004_02_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2004_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2004_05_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2004_06_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2004_07_pl.nc

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_11_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_12_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1997_02_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1997_03_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1997_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1997_05_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1997_06_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1997_07_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1997_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1997_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1997_10_pl.nc

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2009_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2009_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2009_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2009_11_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2009_12_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2010_01_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2010_02_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2010_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2010_05_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2010_06_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2010_07_pl.nc

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2002_01_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2002_02_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2002_03_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2002_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2002_05_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2002_06_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2002_07_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2002_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2002_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2002_11_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2002_12_pl.nc

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1995_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1995_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1995_11_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1995_12_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_01_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_02_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_03_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_05_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_06_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_07_pl.nc